## 3. Классификация текстов
Будем классифицировать на два класса, различать резко отрицательные отзывы (с оценкой 1) и положительные отзывы (с оценкой 5)

In [1]:
import json

import bz2
import regex
from tqdm import tqdm
from scipy import sparse

In [2]:
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [4]:
responses = []
with bz2.BZ2File('banki_responses.json.bz2', 'r') as thefile:
    for row in tqdm(thefile):
        resp = json.loads(row)
        if not resp['rating_not_checked'] and (len(resp['text'].split()) > 0):
            responses.append(resp)

201030it [01:02, 3200.90it/s]


### 3.1. Составьте обучающее и тестовое множество: выберите из всего набора данных N1 отзывов с оценкой 1 и N2 отзывов с оценкой

Используйте sklearn.model_selection.train_test_split для разделения множества отобранных документов на обучающее и тестовое

In [5]:
#наконец-то я созрела перестать делать списки списков со списками внутри и решила нормально через датафрейм всё обработать
#наверное надо было так с самого начала и делать...
df = pd.DataFrame(responses)
df.head(5)

,city,rating_not_checked,title,num_comments,bank_license,author,bank_name,datetime,text,rating_grade
0,г. Москва,False,Жалоба,0,лицензия № 2562,uhnov1,Бинбанк,2015-06-08 12:50:54,Добрый день! Я не являюсь клиентом банка и пор...,NaN
1,г. Новосибирск,False,Не могу пользоваться услугой Сбербанк он-лайн,0,лицензия № 1481,Foryou,Сбербанк России,2015-06-08 11:09:57,Доброго дня! Являюсь держателем зарплатной кар...,NaN
2,г. Москва,False,Двойное списание за один товар.,1,лицензия № 2562,Vladimir84,Бинбанк,2015-06-05 20:14:28,Здравствуйте! Дублирую свое заявление от 03.0...,NaN
3,г. Ставрополь,False,Меняют проценты комиссии не предупредив и не ...,2,лицензия № 1481,643609,Сбербанк России,2015-06-05 13:51:01,Добрый день!! Я открыл расчетный счет в СберБа...,NaN
4,г. Челябинск,False,Верните денежные средства за страховку,1,лицензия № 2766,anfisa-2003,ОТП Банк,2015-06-05 10:58:12,"04.03.2015 г. взяла кредит в вашем банке, заяв...",NaN


In [7]:
import pymorphy3
m = pymorphy3.MorphAnalyzer()

In [8]:
from nltk.corpus import stopwords
import re

In [9]:
mystopwords = stopwords.words('russian') + [
    'это', 'наш' , 'тыс', 'млн', 'млрд', 'также',  'т', 'д',
    'который','прошлый','сей', 'свой', 'наш', 'мочь', 'такой'
]
ru_words = re.compile("[А-Яа-яЁё]+")


def words_only(text):
    return " ".join(ru_words.findall(text))


def lemmatize(text, mystem=m):
    try:
        return " ".join([m.parse(w)[0].normal_form for w in text.split(' ')]).strip()  
    except:
        return " "

def remove_stopwords(text, mystopwords = mystopwords):
    try:
        return " ".join([token for token in text.split() if not token in mystopwords])
    except:
        return ""

    
def preprocess(text):
    return remove_stopwords(lemmatize(words_only(text.lower())))

##### Отбираем в датафрейме классы с 1 и 5, дальше предсказываем на 30 значениях.

In [10]:
list_of_values = [1,5]
df_1_5 = df[df['rating_grade'].isin(list_of_values)]
df_1_5

,city,rating_not_checked,title,num_comments,bank_license,author,bank_name,datetime,text,rating_grade
19,г. Москва,False,Некомпетентность сотрудников банка,2,лицензия № 1481,agelun,Сбербанк России,2015-06-05 15:00:54,Открыт вклад и счет в USD. Плюс к этому есть з...,1.0
33,г. Ростов-на-Дону,False,Навязывание страховки,1,лицензия № 1481,Joker888,Сбербанк России,2015-06-05 21:35:40,Доброго времени! Вчера мне поступило смс-уведо...,1.0
76,г. Санкт-Петербург,False,Нарушение договора банковского вклада,0,лицензия № 3006,qwarta45,АктивКапитал Банк,2015-06-06 23:42:39,"05.06.2015г. около 15 часов, пришел в указанны...",1.0
85,г. Москва,False,"Невозможно оплатить отчисления на капремонт ""Т...",4,лицензия № 1481,721,Сбербанк России,2015-06-04 13:27:04,Для оплаты коммунальных платежей пользуюсь пла...,1.0
94,г. Чапаевск,False,Некомпетентность сотрудника Сбербанка при пров...,16,лицензия № 1481,usmanovaket,Сбербанк России,2015-06-03 16:00:12,В апреле этого года пришла в отделение сбербан...,1.0
...,...,...,...,...,...,...,...,...,...,...
153493,None,False,Два конфликта с банком.,0,лицензия № 2771,Sam,Юниаструм Банк,2005-05-18 01:25:00,Оформил в банке кредитную карту в январе 2004 ...,1.0
153494,None,False,Не все так страшно,0,лицензия № 2557,Вера,Ситибанк,2005-05-18 14:38:00,"Слишком большой банк, не всегда учитывает нашу...",5.0
153496,None,False,"Филиал банка ""ОВК"" в г. Иваново",1,лицензия № 2272,AlexU-post,Росбанк,2005-04-08 14:07:00,Уважаемое руководство банка «ОВК»! Я проживаю ...,1.0
153497,None,False,В Альфа-Банк больше обращаться не буду.,0,лицензия № 1326,Ирина,Альфа-Банк,2005-04-14 15:16:00,Пробовала 10.04.05 оформить товар в кредит в А...,1.0


In [20]:
df_1_5_small = df_1_5.head(30)

In [22]:
%%time
df_1_5_small.text = df_1_5_small.text.apply(preprocess)

CPU times: total: 828 ms
Wall time: 835 ms


<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [24]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_1_5_small.text, df_1_5_small.rating_grade, 
                                                    test_size=0.33, random_state=42) #на маленьких данных большая 
                                                                                     #диспропорция 1 и 5. что-то подсказывает,
                                                                                     #что на больших данных также будет

In [26]:
print(df.shape)
print(X_train.shape)
print(X_test.shape)

(153499, 10)
(20,)
(10,)


### 3.2. Используйте любой известный вам алгоритм классификации текстов для решения задачи и получите baseline.

Для первых 30:

In [29]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report

In [30]:
vec = CountVectorizer(ngram_range=(1,1))
bow = vec.fit_transform(X_train)

In [31]:
clf = LogisticRegression(random_state=7, solver='liblinear')
clf.fit(bow, y_train)

LogisticRegression(random_state=7, solver='liblinear')

In [32]:
pred = clf.predict(vec.transform(X_test))
print(classification_report(pred,y_test))

              precision    recall  f1-score   support

         1.0       1.00      0.80      0.89        10
         5.0       0.00      0.00      0.00         0

    accuracy                           0.80        10
   macro avg       0.50      0.40      0.44        10
weighted avg       1.00      0.80      0.89        10



C:\Users\Алехандро\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Алехандро\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Алехандро\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##### Попробую забалансить классы. Заодно обучу не на 30, а на 2000 значениях. Дальше также оставим по 2000, чёт у меня не хватает терпения предобработки этого всего ждать.

In [28]:
df_1 = df[df['rating_grade'] == 1]
df_5 = df[df['rating_grade'] == 5]

In [30]:
df_1 = df_1.head(2000)
df_5 = df_5.head(2000)

In [32]:
new_df = pd.concat([df_1, df_5])

In [34]:
%%time
new_df.text = new_df.text.apply(preprocess)

CPU times: total: 2min 15s
Wall time: 2min 16s


In [35]:
X_train, X_test, y_train, y_test = train_test_split(new_df.text, new_df.rating_grade, 
                                                    test_size=0.33, random_state=42) #на маленьких данных большая 
                                                                                     #диспропорция 1 и 5. что-то подсказывает,
                                                                                     #что на больших данных также будет

In [36]:
print(df.shape)
print(X_train.shape)
print(X_test.shape)

(153499, 10)
(2680,)
(1320,)


In [37]:
vec = CountVectorizer(ngram_range=(1,1))
bow = vec.fit_transform(X_train)

In [38]:
clf = LogisticRegression(random_state=7, solver='liblinear')
clf.fit(bow, y_train)

LogisticRegression(random_state=7, solver='liblinear')

In [39]:
pred = clf.predict(vec.transform(X_test))
print(classification_report(pred,y_test))

              precision    recall  f1-score   support

         1.0       0.95      0.93      0.94       705
         5.0       0.92      0.94      0.93       615

    accuracy                           0.93      1320
   macro avg       0.93      0.93      0.93      1320
weighted avg       0.93      0.93      0.93      1320



### 3.3. Сравните разные варианты векторизации текста: использование только униграм, пар или троек слов или с использованием символьных 𝑛 -грам.

###### Для первых 30 текстов.

Биграммы:

In [40]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report

In [41]:
vec = CountVectorizer(ngram_range=(2,2))
bow = vec.fit_transform(X_train)

In [42]:
clf = LogisticRegression(random_state=7, solver='liblinear')
clf.fit(bow, y_train)

LogisticRegression(random_state=7, solver='liblinear')

In [43]:
pred = clf.predict(vec.transform(X_test))
print(classification_report(pred,y_test))

              precision    recall  f1-score   support

         1.0       1.00      0.80      0.89        10
         5.0       0.00      0.00      0.00         0

    accuracy                           0.80        10
   macro avg       0.50      0.40      0.44        10
weighted avg       1.00      0.80      0.89        10



C:\Users\Алехандро\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Алехандро\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Алехандро\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Триграммы:

In [44]:
vec = CountVectorizer(ngram_range=(3,3))
bow = vec.fit_transform(X_train)

In [45]:
clf = LogisticRegression(random_state=7, solver='liblinear')
clf.fit(bow, y_train)

LogisticRegression(random_state=7, solver='liblinear')

In [46]:
pred = clf.predict(vec.transform(X_test))
print(classification_report(pred,y_test))

              precision    recall  f1-score   support

         1.0       1.00      0.80      0.89        10
         5.0       0.00      0.00      0.00         0

    accuracy                           0.80        10
   macro avg       0.50      0.40      0.44        10
weighted avg       1.00      0.80      0.89        10



C:\Users\Алехандро\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Алехандро\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Алехандро\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Символьные n-граммы

In [47]:
vec = CountVectorizer(analyzer='char')
bow = vec.fit_transform(X_train)

In [48]:
clf = LogisticRegression(random_state=7, solver='liblinear')
clf.fit(bow, y_train)

LogisticRegression(random_state=7, solver='liblinear')

In [49]:
pred = clf.predict(vec.transform(X_test))
print(classification_report(pred,y_test))

              precision    recall  f1-score   support

         1.0       1.00      0.80      0.89        10
         5.0       0.00      0.00      0.00         0

    accuracy                           0.80        10
   macro avg       0.50      0.40      0.44        10
weighted avg       1.00      0.80      0.89        10



C:\Users\Алехандро\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Алехандро\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Алехандро\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


###### Для 2000 текстов.

Биграммы:

In [40]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report

In [41]:
vec = CountVectorizer(ngram_range=(2,2))
bow = vec.fit_transform(X_train)

In [42]:
clf = LogisticRegression(random_state=7, solver='liblinear')
clf.fit(bow, y_train)

LogisticRegression(random_state=7, solver='liblinear')

In [43]:
pred = clf.predict(vec.transform(X_test))
print(classification_report(pred,y_test))

              precision    recall  f1-score   support

         1.0       0.91      0.89      0.90       704
         5.0       0.87      0.90      0.88       616

    accuracy                           0.89      1320
   macro avg       0.89      0.89      0.89      1320
weighted avg       0.89      0.89      0.89      1320



Триграммы:

In [45]:
vec = CountVectorizer(ngram_range=(3,3))
bow = vec.fit_transform(X_train)

In [46]:
clf = LogisticRegression(random_state=7, solver='liblinear')
clf.fit(bow, y_train)

LogisticRegression(random_state=7, solver='liblinear')

In [47]:
pred = clf.predict(vec.transform(X_test))
print(classification_report(pred,y_test))

              precision    recall  f1-score   support

         1.0       0.64      0.86      0.74       509
         5.0       0.89      0.69      0.78       811

    accuracy                           0.76      1320
   macro avg       0.77      0.78      0.76      1320
weighted avg       0.79      0.76      0.76      1320



Символьные n-граммы:

In [49]:
vec = CountVectorizer(analyzer='char')
bow = vec.fit_transform(X_train)

In [50]:
clf = LogisticRegression(random_state=7, solver='liblinear')
clf.fit(bow, y_train)

LogisticRegression(random_state=7, solver='liblinear')

In [51]:
pred = clf.predict(vec.transform(X_test))
print(classification_report(pred,y_test))

              precision    recall  f1-score   support

         1.0       0.74      0.75      0.74       678
         5.0       0.73      0.72      0.73       642

    accuracy                           0.73      1320
   macro avg       0.73      0.73      0.73      1320
weighted avg       0.73      0.73      0.73      1320



Униграммы:

In [71]:
vec = CountVectorizer()
bow = vec.fit_transform(X_train)

In [73]:
clf = LogisticRegression(random_state=7, solver='liblinear')
clf.fit(bow, y_train)

LogisticRegression(random_state=7, solver='liblinear')

In [75]:
pred = clf.predict(vec.transform(X_test))
print(classification_report(pred,y_test))

              precision    recall  f1-score   support

         1.0       0.95      0.93      0.94       705
         5.0       0.92      0.94      0.93       615

    accuracy                           0.93      1320
   macro avg       0.93      0.93      0.93      1320
weighted avg       0.93      0.93      0.93      1320



Лучше всего сработали униграммы.

## 3.4. Сравните, как изменяется качество решения задачи при использовании скрытых тем в качестве признаков.
1-ый вариант: 𝑡𝑓−𝑖𝑑𝑓 преобразование (sklearn.feature_extraction.text.TfidfTransformer) и сингулярное разложение (оно же – латентый

семантический анализ) (sklearn.decomposition.TruncatedSVD),

In [163]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [175]:
X_train, X_test, y_train, y_test = train_test_split(new_df['text'], new_df['rating_grade'], test_size=0.3, random_state=7)

In [188]:
clf = Pipeline([ 
    ('vect', CountVectorizer(analyzer = 'word', ngram_range=(2,2))), #с char результат хуже
    ('tfidf', TfidfTransformer()), 
    ('tm', TruncatedSVD()), 
    ('clf', RandomForestClassifier())
])

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print("F1:",(f1_score(y_test, y_pred)))
print("ROC: ",(roc_auc_score(y_test, y_pred)))
print("Accuracy: ", (accuracy_score(y_test, y_pred)))
print("Classification report: \n", (classification_report(y_test, y_pred)))

F1: 0.7833876221498371
ROC:  0.7784197602195579
Accuracy:  0.7783333333333333
Classification report: 
               precision    recall  f1-score   support

         1.0       0.76      0.80      0.78       598
         5.0       0.79      0.75      0.77       602

    accuracy                           0.78      1200
   macro avg       0.78      0.78      0.78      1200
weighted avg       0.78      0.78      0.78      1200

